# Lab exercise #3: ELBO and GPLVMs

CS-E4075 2021

# Task 1: Variational inference for Gaussian Process Classification (worth 2 points)

## Learning objectives

After completing this lab exercise, you should be able to:

- Implement Variational inference for GP classification


**/!\ Important Notes /!\**
* In this notebook, we **won't** be implementing sparse GPs (the approximation using inducing points). However, completing this notebook will give you all the tools and building blocks to implement them.
* For speed purposes, it is highly recommended to use an automatic differentiation framework such as tensorflow or pytorch. (optimization using numpy/scipy also works, but will be much slower!). Examples and hints in this notebook are using tensorflow but can be adapted to run in alternative frameworks.
* All exercises must be solved using only basic mathematical operations (exp, erf, ...) and linear algebra routines (solve, matrix-vector products, ...)



**A mini tutorial on automatic differentiation**

When using an automatic differentiation framework to optimize a function $f: \theta \to f(\theta)$, the variable $\theta$ and/or the operations mapping from $\theta$ to $f(\theta)$ must be defined using operators from the framework.

For example to optimize $e^{\theta}+e^{-\theta}$ with respect to $\theta$ with tensorflow, you need to proceed as follows:


In [ ]:
import tensorflow as tf

# define the theta variable
theta = tf.Variable(1.0, dtype=tf.float64)

# define the function
f = lambda x: tf.exp(x) + tf.exp(-x) # note the use of the tf.exp operation (not np.exp)

# run the optimization
for t in range(1000):
    # at each step, compute the gradients
    with tf.GradientTape() as tape:
        tape.watch(theta)
        loss = f(theta)
    
    gradient = tape.gradient(loss, theta)
    
    # apply the variable update (gradient descent)
    theta.assign(theta - 0.01*gradient)
    
    if t % 100 == 0:
        print(t, theta)

In [ ]:
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt

We are interested in the problem of Gaussian Process classification. 

We have some data $\mathcal{D} = \left\lbrace {\bf x}_n, y_n \right\rbrace_{n=1}^N$, with $y_n \in \{-1,1\}$.

We want to perform inference in the following generative model
$$ f \sim GP(0, k)$$
$$ p(y_n=1|{\bf x}_n) = \phi(y_n * f_n),$$
with $\phi$ the normal cumulative distribution function $\phi(x)=\int_{-\infty}^x {\cal N}(u; 0,1)du$.

We will here use a RBF kernel, with two parameters: lengthscale $l$ and variance $\sigma^2$.


The posterior is $p({\bf f}|{\bf y}) \propto p({\bf y}|{\bf f})p({\bf f})$ is intractable, hence we resort to an approximate inference scheme called variational inference.

This turns inference into optimization. We optimize the distance $d(q) = KL[q({\bf f})||p({\bf f}|{\bf y})] \geq 0$, with respect to a distribution $q({\bf f})$

We parameterize $q$ through the mean vector $m$ and the Cholesky factor of the covariance $L$: i.e. $q({\bf f})={\cal N}({\bf f}|m, S=LL^T)$

In practice we optimize the ELBO:
$${\cal L}(q) = \log p({\bf y})-d(q) = 
\underbrace{\mathbb{E}_q \log p({\bf y}|{\bf f})}_{VE} 
- \underbrace{KL(q({\bf f})||p({\bf f}))}_{KL}$$

We split the ELBO into two terms
* variational expectations (VE)
* Kullback Leibler (KL)


### Task 1a: KL divergence

For a prior $p({\bf f})={\cal N}({\bf f}|0,K)$ and a variational distribution $q({\bf f})={\cal N}({\bf f}|m, S=LL^T)$, compute the KL divergence $KL(q({\bf f})||p({\bf f}))$


You can use the formula :
$$
\begin{align*}
&KL\left(\mathcal{N}(\mu_0,\Sigma_0) \parallel \mathcal{N}(\mu_1,\Sigma_1)\right) \\ 
 &= \frac{1}{2}\left(
    \operatorname{tr}\left(\Sigma_1^{-1}\Sigma_0\right) +
    \left(\mu_1 - \mu_0\right)^\mathsf{T} \Sigma_1^{-1}\left(\mu_1 - \mu_0\right) - k +
    \ln\frac{|\Sigma_1|}{|\Sigma_0|}
  \right),\; (source: wikipedia)\\
  &= \dots \quad \text{ (bonus : can you fill the gap?)}\\
    &=
  \frac{1}{2}\left(
    \sum_{ij} (L_1^{-1}L_0)^2_{ij} +
    ||L_1^{-1}\left(\mu_1 - \mu_0\right)||^2 - k + 2\sum_{i}
    \ln |L_{1,ii}|- \ln|L_{0,ii}|
  \right).
  \end{align*}
  $$

**Note**: this needs to be adapted to the (mean,cholesky) parameterization of the multivariate Gaussian distributions.


In [ ]:
def KL(m0, L0, m1, L1):
    """ returns the KL divergence between N(m0, S0) and N(m1, S1)
    
    arguments:
    m0, m1   -- N x 1, mean vector
    L0, L1   -- N x N, Cholesky factor of a covariance matrix 
    returns a scalar
    """
    
    ###############################################
    # ------- insert code here -------------------
    ###############################################
    


Let's check that the KL is coded properly.

For instance, noting $q_0(f) = N(f|0, I)$ and $q_1(f) = N(f|0, 2I)$, 
we should have:
* $KL[q_0||q_0] = 0$
* $KL[q_0||q_1] > 0$ 



In [ ]:
K = 10
m_0 = m_1 = np.zeros((K,1))
L_0 = np.eye(K)
L_1 = np.sqrt(2.) * np.eye(K)

assert KL(m_0, L_0, m_0, L_0) == 0
assert KL(m_0, L_0, m_1, L_1) >= 0

print(KL(m_0, L_0, m_0, L_0))
print(KL(m_0, L_0, m_1, L_1))

### Task 1b: Variational expectations

To compute the variational expectations $\mathbb{E}_{q(f_n)} \log p(y_n|f_n)$, we first need to compute the marginal distribution $q(f_n)$ and then compute the expectation.


In [ ]:
def q_marginals(m, L):
    """ returns the vectors of marginal means and marginal variances
    i.e, the means and variances of q(f_n)
    
    Hint: You may want to use the tf.reduce_sum
    
    arguments:
    m   -- N x 1, mean vector
    L   -- N x N, Cholesky factor of a covariance matrix 
    returns : 2 N x 1 vectors
    """
    
    ###############################################
    # ------- insert code here -------------------
    ###############################################



In [ ]:
def phi(x):
    """ Cumulative distribution function for the standard normal distribution 
    Hint: you may want to use the error function. (tf.math.erf if using tensorflow)

    phi(x) = int_{-\infty, x} N(u| 0, 1) du    
    """
    ###############################################
    # ------- insert code here -------------------
    ###############################################
    
def classif_log_likelihood(f, y):
    """ log p(y|f) for classification using the normal cdf 
        log p(y|f) = log phi(y * f)
    """
    ###############################################
    # ------- insert code here -------------------
    ###############################################


    
# --------------------------------------
# The next function is given to you.
# It approximates E_q(f_n) log p(y_n|f_n) via Gaussian quadrature
# see: https://en.wikipedia.org/wiki/Gauss%E2%80%93Hermite_quadrature
# --------------------------------------
def expected_log_likelihood(
    means, covs, llh, y, n_gh=10):
    """ returns the expected log likelihood terms
    
    E_q(f_n) log p(y_n|f_n)
    
    This is a quadrature approximation, 
    turning the integral into a sum.
        
    arguments:
    means  -- N x 1, vector of means
    covs  -- N x 1, vector of covariances 
    llh -- log likelihood function
    y  -- N x 1, vector of observed labels 
    """
    z, dz = np.polynomial.hermite.hermgauss(n_gh)
    weights = (dz / np.sqrt(np.pi)).reshape(1, -1) # 1 x n_gh 
    inputs = means + np.sqrt(2 * covs) * z.reshape(1, -1) # N x n_gh
    llh_quad = weights * llh(inputs, y) # N x n_gh

    # 'tf.reduce_sum' is tensorflow's summing function, 
    # replace if using another framework  
    return tf.reduce_sum(llh_quad, axis=1) # N, 

### Task 1c: ELBO

We are now ready to implement the ELBO as the difference between the variational expectations and the KL divergence:

$${\cal L}(q) = 
\underbrace{\mathbb{E}_q \log p({\bf y}|{\bf f})}_{VE} 
- \underbrace{KL(q({\bf f})||p({\bf f}))}_{KL}$$




In [ ]:
def elbo(m_p, L_p, m_q, L_q, y):
    """ returns ELBO
    L = \sum_n E_q(f_n) log p(y_n|f_n)
        + KL(q(f)||p(f))
        
    (See slides of lecture 4 for closed form solution)
    
    arguments:
    L_p, L_q  -- N x N, Cholesky factors of the covariances of p and q
    m_p, m_q -- N x 1, mean vector of p and q
    returns: a scalar
    """
    
    ###############################################
    # ------- insert code here -------------------
    ###############################################


    
    

### Task 1d: Inference as optimization

We are now ready to optimize the ELBO.
We will first load some data

In [ ]:
# Loading the data

import csv
XY = []
with open("banana.csv") as csvfile:
    reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC) # change contents to floats
    for row in reader: # each row is a list
        XY.append(row)
XY = np.array(XY)

# Here we select a subset of the data. (remember computations scales as N^3)
N = 50
X, Y = XY[:N,:-1],XY[:N,-1:]
Y = (Y-1.5) * 2 # to be in {-1, 1}
N = X.shape[0]

# Plotting the data

plt.scatter(X[:,0], X[:,1], c=Y)
plt.xlabel('$x_1$', fontsize=15)
plt.ylabel('$x_2$', fontsize=15)
plt.title('Classification data', fontsize=20);

#### Preparing prior statistics

We need to compute the prior covariance $K_p = K_{\bf ff}$ and its Cholesky factor $L_p = chol(K_{\bf ff})$.

In [ ]:
# kernel parameters
l = 0.5
s = 0.5 # the standart deviation

### computing the kernel matrix K_ff

###############################################
# ------- insert code here -------------------
###############################################


### Computing m_p, L_p = cholesky(K_p).

###############################################
# ------- insert code here -------------------
###############################################



We initialize the variational distribution to $q({\bf f})={\cal N}({\bf f};0, I)$,
then optimize the ELBO using gradient based optimization.


Gradient based optimization refers to optimization schemes where a function $f(\theta)$ is optimized with respect to $\theta$ by following the gradient $\nabla_{\theta} f(\theta)$.
For example gradient descent construct a sequence of values $\theta_t$ following
$$\theta_{t+1 } = \theta_t + \eta \nabla_{\theta} f(\theta)|_{\theta=\theta_t}$$
where $\eta$ is the learning rate.


When using an automatic differentiation framework, one does not need to manually derive the gradient (hence the 'automatic'). Such frameworks include tensorflow, jax, pytorch (pick your favorite). These are widely used to optimize the loss function of neural network models in supervised learning.

In [ ]:
# initial distribution parameters m_q, L_q
m_q = tf.Variable(np.zeros((N, 1)), dtype=tf.float64)
L_q = tf.Variable(np.eye(N), dtype=tf.float64)

# Optimize the loss: a tensorflow routine is given
loss = lambda m, L: - elbo(m_p, L_p, m, L, Y)

# definition of a training step
def train(opt, m, L):
    with tf.GradientTape() as tape:
        tape.watch([m, L])
        loss_ = - elbo(m_p, L_p, m, L, Y)
    gradients = tape.gradient(loss_, [m, L])
    opt.apply_gradients(zip(gradients, [m, L]))

# you can change the optimizer or learning rate
opt = tf.optimizers.Adam(learning_rate=.0001) 

# running the optimization
for t in range(5000):
    train(opt, m_q, L_q)
    if t % 500 == 0:
        print(t, loss(m_q, L_q))   


* Plot the evolution of the ELBO as a function function of iterations.

* Plot the posterior process $p(f^*|x^*, {\cal D})$.

* Plot the predictive distribution $p(y^*=1|x^*)$.

* Repeat the procedure for different values of $(\sigma^2, l)$, can you see an improvement? Is the ELBO a good proxy for hyperparameter optimization in this example?


In [ ]:
###############################################
# ------- insert code here -------------------
###############################################




### Task 1e: Posterior prediction for new data points

Under the hood, the algorithm defines a posterior process for all values of the input space.

For a new input $x^*$, the posterior prediction is given by 

$
\begin{align*}
q(f(x^*)) &= \int p(f(x^*)|{\bf f})q({\bf f})d{\bf f}\\
 &= {\cal N}(f(x^*)| K_{f^*{\bf f} }K_{{\bf ff}}^{-1} m_q,
 K_{f^*f^*} - K_{f^*{\bf f}}K_{{\bf ff}}^{-1}(K_{{\bf ff}} - S)K_{{\bf ff}}^{-1}K_{{\bf f} f^*})
\end{align*}
$

In [ ]:
def posterior_marginal_prediction(X_new, X, m_q, L_q):
  """ compute the posterior marginal predictions q(f(x*)) 
  independently for all inputs in X_new 
  
  Note: You need to now use tensorflow functions
  
  arguments:
    X_new -- N_new x 2, matrix of new inputs
    X     -- N x 2, matrix of training inputs
    L_q   -- N x N, Cholesky factor of the covariances of q
    m_q   -- N x 1, mean vector of q
    returns: predictive marginal means and variances (both with size N_new x 1) 
  """


  ###############################################
  # ------- insert code here -------------------
  ###############################################

    
    
    

Plotting the prediction

In [ ]:
# create new input points on grid
n_grid = 100
x = np.linspace(X.min(), X.max(), n_grid)
X1new, X2new = np.meshgrid(x, x)
Xnew = np.hstack([
  X1new.reshape(-1,1), X2new.reshape(-1,1)
])  # size : n_grid * n_grid x 2

###############################################
# ------- insert code here -------------------
###############################################
  

    

### Advanced [for the curious, no extra points]
* Repeat the procedure for the regression setting with Gaussian noise. You need to derive new variational expectations since the likelihood changes. Apply the resulting algorithm to the regression problem of the previous assignment.
* For fixed hyperparameters, do the ELBO match the marginal likelihood $\log p({\bf y})$? If so why?

In [ ]:
###############################################
# ------- insert code here -------------------
###############################################

# Task 2: GPLVM's (worth 1 point)

Latent variable models attempt to capture hidden structure in high dimensional
data. Given a collection of
high-dimensional observations (e.g., images), we can posit some low-dimensional
latent structure. We assume that, conditional on the latent structure, the large
number of outputs (pixels in the image) are independent of each other. Training
in this model consists of
   1. optimizing model parameters (kernel function parameters as well as, e.g.,
   observation noise variance), and
   2. finding, for each training observation (image), a corresponding point
   location in the index set.
All of the optimization can be done by maximizing the marginal log likelihood of
the data.

## Imports

For these tasks you Tensorflow, gpflow and GPy libraries. 

In [ ]:
import numpy as np
np.random.seed(1)  # for reproducibility
import matplotlib.pyplot as plt

import tensorflow as tf
import gpflow
from gpflow.utilities import ops, print_summary
from gpflow.config import set_default_float, default_float, set_default_summary_fmt
from gpflow.ci_utils import ci_niter

from sklearn.decomposition import PCA

%pylab inline
%matplotlib inline

set_default_float(np.float64)
set_default_summary_fmt("notebook")

In [ ]:
def load_MNIST(N=500):
    import random

    (y_train, labels_train), (_, _) = tf.keras.datasets.mnist.load_data()

    # Shuffle data and subsample
    new_idx = np.arange(y_train.shape[0])
    np.random.shuffle(new_idx)
    y_train, labels_train = y_train[new_idx, :, :], labels_train[new_idx]
    sub_y_train = y_train[:N, ...].astype(np.float64) / 256.
    labels = labels_train[:N]
    y = sub_y_train.reshape(N, -1)

    def view_MNIST():
        # Lets look at the sub sampled data
        rand_idx = np.random.randint(0, N-1)
        plt.imshow(y[rand_idx, :].reshape((28,28)), interpolation='none', cmap='Greys')
        plt.title(f'Random sample with label {labels[rand_idx]}')
        plt.show()

    view_MNIST()

    print("Number of points: {} and Number of dimensions: {}".format(y.shape[0], y.shape[1]))
    return y, labels

def load_three_phase_oil():
    data = np.load("./data/three_phase_oil_flow.npz")
    y =data["Y"]
    labels = data["labels"]
    
    print("Number of points: {} and Number of dimensions: {}".format(y.shape[0], y.shape[1]))
    return y, labels

def load_swiss_roll(N=500):
    from sklearn import datasets
    y, color = datasets.make_swiss_roll(n_samples=N)
    
    print("Number of points: {} and Number of dimensions: {}".format(y.shape[0], y.shape[1]))
    return y, color

def load_decampos_digits():
    import GPy
    which = [0,1,2,6,7,9] # which digits to work on
    data = GPy.util.datasets.decampos_digits(which_digits=which)
    y = data['Y']
    labels = data['str_lbls'].ravel()
    
    print("Number of points: {} and Number of dimensions: {}".format(y.shape[0], y.shape[1]))
    return y, labels



## (B)GPLVM model setup

In [ ]:
# Shared model parameter set up
#latent_dim = 2                            # number of latent dimensions
#n_data_points = y.shape[0]                # number of data points
#n_data_dims = y.shape[1]                  # number of data dimensions

def create_GPLVM(kernel):
    
    X_init = ops.pca_reduce(y, latent_dim)              # Initialise latent
    
    # alternative initialisations...
        
    X_parameter = gpflow.base.Parameter(X_init)
    Y_tensor = gpflow.models.util.data_input_to_tensor(y)

    gplvm = gpflow.models.gpr.GPR((X_parameter, Y_tensor), kernel=kernel)
    gplvm.likelihood.variance.assign(0.01)
    
    return gplvm

def create_GPLVM_wrapped(kernel):
    # Initialise latent
    X_mean_init = ops.pca_reduce(y, latent_dim)
    
    gplvm = gpflow.models.GPLVM(
        y,
        latent_dim = latent_dim,
        X_data_mean = X_mean_init,
        kernel=kernel,
    )

    gplvm.likelihood.variance.assign(0.01)

    # Helper function
    #def get_latent(model):
    #    return 
    
    return gplvm#, get_latent

def create_BGPLVM(kernel, num_inducing=25):
    # Initialise latent
    X_mean_init = ops.pca_reduce(y, latent_dim)
    
    # Initial inducing points
    inducing_variable = tf.convert_to_tensor(
        np.random.permutation(X_mean_init.numpy())[:num_inducing], dtype=default_float()
    )
    # Initialise latent variance
    X_var_init = tf.ones((y.shape[0], latent_dim), dtype=default_float())


    gplvm = gpflow.models.BayesianGPLVM(
        y,
        X_data_mean=X_mean_init,
        X_data_var=X_var_init,
        kernel=kernel,
        inducing_variable=inducing_variable,
    )

    gplvm.likelihood.variance.assign(0.01)
    
    return gplvm

get_latent_mean = lambda model: model.X_data_mean.numpy()
get_latent = lambda model: model.data[0].numpy()


## GPLVM and comparison to PCA

In [ ]:
y, labels = load_decampos_digits()

# PCA
pca_latent = 12
pca = PCA(n_components=pca_latent)
X_pca = pca.fit_transform(y-y.mean())
f, ax = plt.subplots(1, 2, figsize=(10, 4.8))
ax[0].set_title('First two principal components')
for i in np.unique(labels):
    ax[0].scatter(X_pca[labels == i, 0], X_pca[labels == i, 1], label=i)
ax[1].bar(np.linspace(0, pca_latent, pca_latent), pca.explained_variance_)
ax[1].set_title('Variance explained');
plt.show()

# GPLVM with linear kernel
latent_dim = 4
lengthscales = tf.convert_to_tensor([1.] * latent_dim, dtype=default_float())
kernel = gpflow.kernels.Linear(variance=lengthscales)  
gplvm = create_GPLVM(kernel)
opt = gpflow.optimizers.Scipy()                       
maxiter = ci_niter(10000)
_ = opt.minimize(
    gplvm.training_loss,
    method="BFGS",
    variables=gplvm.trainable_variables,
    options=dict(maxiter=maxiter),
)     
print_summary(gplvm)
order = gplvm.kernel.variance.numpy().argsort()
f, ax = plt.subplots(1, 2, figsize=(10, 4.8))
X_gplvm_linear = gplvm.data[0].numpy()[:, order]
for i in np.unique(labels):
    ax[0].scatter(X_gplvm_linear[labels == i, 0], X_gplvm_linear[labels == i, 1], label=i)
#ax[0].scatter(X_gplvm_linear[:, 0], X_gplvm_linear[:, 1], c=labels)
ax[1].bar(np.linspace(0, latent_dim, latent_dim), 1/gplvm.kernel.variance.numpy()[order])
ax[1].set_title('Variance explained');


## Task 2a

How does your linear solution differ between PCA and GPLVM with a linear kernel? Look at the plots and also try and consider how the linear ARD parameters compare to the eigenvalues of the principal components.


__Solution__

[insert]


## Task 2b

Change the initialisation of the latent variables X_init inside the GPLVM model builder function. How this does change the results?

Hint: Try random noise, or a subset of the dimensions.

__Solution__

[insert]

## Task 2c

The next step is to use a non-linear mapping between latent variables  __$X$__  and features  __$Y$__  by selecting the exponentiated quadratic covariance function. Run the code below.

How does choosing a non-linear kernel affect the results? Are there digits that the GPLVM with an exponentiated quadratic covariance can separate, which PCA is not able to?

__Solution__

[insert]

In [ ]:
lengthscales = tf.convert_to_tensor([1.] * latent_dim, dtype=default_float())
kernel = gpflow.kernels.RBF(lengthscales=lengthscales)
gplvm = create_GPLVM(kernel)

opt = gpflow.optimizers.Scipy()                       
maxiter = ci_niter(1000)
_ = opt.minimize(
    gplvm.training_loss,
    method="BFGS",
    variables=gplvm.trainable_variables,
    options=dict(maxiter=maxiter),
)     
print_summary(gplvm)

order = (gplvm.kernel.lengthscales.numpy()).argsort()
f, ax = plt.subplots(1, 2, figsize=(10, 4.8))
X_gplvm_rbf = gplvm.data[0].numpy()[:, order]
for i in np.unique(labels):
    ax[0].scatter(X_gplvm_rbf[labels == i, 0], X_gplvm_rbf[labels == i, 1], label=i)
ax[0].legend()
ax[1].bar(np.linspace(0, latent_dim, latent_dim), 1/gplvm.kernel.lengthscales.numpy()[order])


## Bayesian GPLVM

GPLVM needs not make anyassumptions on the prior of latent variables. However, lack of such assumption makes the model inferred by just maximizingthe log marginal likelihood prone to overfitting. To tackle this problem, one of effective approaches is to impose a specific prior onto the latent variables for a posterior estimation. Thus, we can introduce various constraints into the prior for the estimation the latent variables in different tasks. Specifically, we assume that $p(X)$ denotes the imposed prior. By using the Bayesian theorem, we can formulate the posterior probability of the latent variables X as $p(X|Y,\theta) \propto p(Y|X,\theta)p(X)$ 



In [ ]:
lengthscales = tf.convert_to_tensor([1.] * latent_dim, dtype=default_float())
kernel = gpflow.kernels.RBF(lengthscales=lengthscales)
bgplvm = create_BGPLVM(kernel, num_inducing=20)

# This will take minutes to run.
# You can interrupt the kernel
opt = gpflow.optimizers.Scipy()                       
maxiter = ci_niter(1000)
_ = opt.minimize(
    bgplvm.training_loss,
    method="BFGS",
    variables=bgplvm.trainable_variables,
    options=dict(maxiter=maxiter),
)     
print_summary(bgplvm)

In [ ]:
order = (bgplvm.kernel.lengthscales.numpy()).argsort()
f, ax = plt.subplots(1, 2, figsize=(10, 4.8))
X_bgplvm_rbf = bgplvm.X_data_mean.numpy()[:, order]
for i in np.unique(labels):
    ax[0].scatter(X_bgplvm_rbf[labels == i, 0], X_bgplvm_rbf[labels == i, 1], label=i)
ax[0].legend()
ax[1].bar(np.linspace(0, latent_dim, latent_dim), 1/bgplvm.kernel.lengthscales.numpy()[order]);


## Task 2d 

* How does the Bayesian GP-LVM compare with the GPLVM model?  
* How has the prior on $X$ affected the results?
* Are there any classes that still overlap? Why?

__Solution__

* ...
* ...
* ...